In [56]:
#####################################################################################################################
##### Python modules
import os, json, h5py
import numpy as np

##### OpenSRA modules and functions
from src import fcn_gen

##### SimCenter modules
import lib.simcenter.OpenSHAInterface as OpenSHAInterface

In [22]:
save_dir = r'C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\lib\simcenter\ucerf3_1'

In [2]:
reg_proc = OpenSHAInterface.initEQHazardInterface(1)

In [3]:
##
nSources = reg_proc.getNumSources()

In [4]:
nSources

261031

In [5]:
#Initializing the interface to EQHazardCalc
rupSet, rupSourceSections = OpenSHAInterface.set_up_get_rup()

In [6]:
seg_unique = {}
ps_unique = {}
src_connect = {}

for src_i in range(nSources):
    if src_i%10000==0:
        print(src_i)
    try:
        ## get list of segments
        listSeg = list(rupSourceSections[src_i].toArray())
        src_connect.update({str(src_i):listSeg})

        ## get list of nodes for all segments in current source
        for j in range(len(listSeg)):
            if not str(listSeg[j]) in seg_unique:
                seg_unique[str(listSeg[j])] = {}
                section = rupSet.getFaultSectionData(listSeg[j])
                trace = section.getFaultTrace()
                nodes = []
                for point in trace:
                    nodes.append([float(point.getLongitude()),float(point.getLatitude()),float(point.getDepth())])
                ##
                seg_unique[str(listSeg[j])].update({'name':str(section.getSectionName())})
                seg_unique[str(listSeg[j])].update({'trace':nodes})

    except:
        ## point sources
        reg_proc.setCurrentRupture(src_i,0)
        rupture = reg_proc.getRupture()
        surface = rupture.getRuptureSurface()
        nodes = [[float(surface.getLocation().getLongitude()), 
                    float(surface.getLocation().getLatitude()), 
                    float(surface.getLocation().getDepth())]]
        ps_unique.update({str(src_i):nodes[0]})

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000


In [23]:
src_connect_arr = []
for i in src_connect:
    src_connect_arr.append([i,src_connect[i]])
    
f = open(os.path.join(save_dir,'src_connect.txt'), "w")
for line in src_connect_arr:
    # write line to output file
    f.write(str(line))
    f.write("\n")
f.close()

In [24]:
with open(os.path.join(save_dir,'rup_seg.json'), "w") as f:
    json.dump(seg_unique, f, indent=4)

In [26]:
ps_unique_arr = []
for i in ps_unique:
    ps_unique_arr.append([i,ps_unique[i][0],ps_unique[i][1],ps_unique[i][2]])

np.savetxt(os.path.join(save_dir,'point_source.txt'),ps_unique_arr,fmt='%s')

In [100]:
list_pt_src = np.loadtxt(r'C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\lib\simcenter\ucerf3_1\point_source.txt')

In [96]:
np.where(list_pt_src[0] ==  253395)[0][0]

0

In [104]:
count = 0
for pt_src in list_pt_src:
    count += 1
    if count < 2:
        print(pt_src[1:3])

[ 2.53395e+05 -1.17200e+02  3.16000e+01  0.00000e+00]


In [3]:
with open(os.path.join(r'C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\lib\simcenter\ucerf3_1','rup_seg.json'), 'r') as read_file:
    list_rup_seg = json.load(read_file)
read_file.close()

In [6]:
list_rup_seg['10']

{'name': 'La Panza 2011, Subsection 3',
 'trace': [[-120.14930992251927, 35.20224071795313, 0.0],
  [-120.1879, 35.23274, 0.0],
  [-120.21786851991303, 35.24637630411823, 0.0]]}

In [66]:
count = 0
seg_pass_rmax = []
for seg in list_rup_seg:
    count+=1
    flag_rmax = check_rmax([[-120,35],[-110,30]],list_rup_seg[seg]['trace'],100)
    if flag_rmax == True:
        seg_pass_rmax.append(int(seg))

In [67]:
len(seg_pass_rmax)

298

In [68]:
seg_pass_rmax[:10]

[1305, 2346, 2364, 1801, 784, 1547, 1679, 231, 2173, 1732]

In [57]:
## initialize dictionaries
gm_source_info = {'src':None,'rup':None,'M':None,'rate':None}
keys = list(gm_source_info)

## see if rup_meta_file already exists
## load rup_meta_file
with h5py.File(r'C:\Users\barry\Desktop\opensra_results_for_chris\gm_out\rup_meta_10000yr.hdf5', 'r') as f:
    for key in keys:
        gm_source_info[key] = f.get(key)[:]
f.close()

In [61]:
src_unique = np.unique(gm_source_info['src'])

In [59]:
#####
reg_proc = OpenSHAInterface.initEQHazardInterface(1)

In [63]:
_, rupSourceSections = OpenSHAInterface.set_up_get_rup()

In [80]:
#####
count = 0
for src_unique_i in src_unique:
    count+=1
    if count <= 10:
        listSeg = list(rupSourceSections[src_unique_i].toArray())
        print(common_member(listSeg,seg_pass_rmax))

No common elements
None
{2, 2439, 1801, 1547, 2189, 1679, 784, 1297, 1305, 282, 1188, 2346, 940, 814, 823, 441, 1595, 2364, 1732, 836, 1992, 989, 231, 1906, 1781, 2040, 2298, 2173, 1662}
None
No common elements
None
No common elements
None
No common elements
None
No common elements
None
No common elements
None
No common elements
None
No common elements
None
No common elements
None


In [78]:
listSeg

[2447, 1495, 520, 1215, 866, 1419, 496, 400, 2504, 2512, 1127, 180]

In [69]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
  
    if (a_set & b_set): 
        print(a_set & b_set) 
    else: 
        print("No common elements")  

In [45]:
#####################################################################################################################
##### check for list of rupture segments that are within rmax
#####################################################################################################################
def check_rmax(site_loc,seg_trace,rmax):
    """
    check for list of rupture segments that are within rmax
    
    """
    ##
    for i in range(len(seg_trace)-1):
        seg_sub = [seg_trace[i][:2],seg_trace[i+1][:2]]
        ##
        for site_j in site_loc:
            ##
            _,dist = get_closest_pt(site_j,seg_sub)
            ##
            if dist <= rmax:
                ##
                return True
                
    return False

In [53]:
#####################################################################################################################
##### get closest projection and spherical distance for a location relative to a line segment
#####################################################################################################################
def get_closest_pt(loc,line):
    """
    returns the closest location and shortest distance between a location and a line segment

    Parameters
    ----------
    loc : [float, float]
        [degree] coordinate of point
    line : [[float, float], [float, float]]
        [degree] two pairs of coordinates [pt1, pt2] that define the line
    
    Returns
    -------
    closest_loc : [float, float]
        [degree] closest point on line
    shortest_dist : [float, float]
        [degree] shorest distance
    
    """
    
    ## calculations
    line_vect = [line[1][0]-line[0][0],line[1][1]-line[0][1]] # get vector for line from pt 1 to pt 2
    len_line = np.dot(line_vect,line_vect)**0.5 # length of line
    if line_vect[0] == 0:
        line_theta = np.radians(90)
    else:
        line_theta = np.arctan(line_vect[1]/line_vect[0]) # rad, angle for line vector relative to horizontal
    loc_vect = [loc[0]-line[0][0],loc[1]-line[0][1]] # get vector from pt 1 of line to pt to project
    proj_dist = np.dot(loc_vect,line_vect/len_line) # compute projected length of pt on line
    
    ## check if point is within or outside of line:
    if proj_dist <= 0:
        closest_loc = line[0]
    elif proj_dist >= len_line:
        closest_loc = line[1]
    else:
        closest_loc = [line[0][0]+proj_dist*np.cos(line_theta),
                        line[0][1]+proj_dist*np.sin(line_theta)] # compute projection
        closest_loc = np.round(closest_loc,decimals=6)
    
    ##
    shortest_dist = get_haversine_dist(loc[0],loc[1],closest_loc[0],closest_loc[1])
    
    ##
    return closest_loc,shortest_dist

In [10]:
#####################################################################################################################
##### Haversine equation for distances
#####################################################################################################################
def get_haversine_dist(lon1,lat1,lon2,lat2,unit='km'):
	"""
	calculates the Haversine distance between two sets of coordinates
	
	Parameters
	----------
	lon1 : float, array
		[degree] longitude of site 1
	lat1 : float, array
		[degree] latitude of site 1
	lon2 : float, array
		[degree] longitude of site 2
	lat2 : float, array
		[degree] latitude of site 2
	unit : str
		unit for output: '**km**' or '**miles**'
	
	Returns
	-------
	
	d : float, array
		distance
	
	"""
	
	## determine unit to reference for Earth's radius
	if 'k' in unit.lower():
		r = 6371 # km
	elif 'mi' in unit.lower():
		r = 3958.8 # miles
	
	## convert long lat from degrees to radians
	lon1 = np.radians(lon1)
	lat1 = np.radians(lat1)
	lon2 = np.radians(lon2)
	lat2 = np.radians(lat2)
	
	## Haversine function for epicentral distance
	d = 2*r*np.arcsin(np.sqrt(np.sin((lat2-lat1)/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin((lon2-lon1)/2)**2))
	
	##
	return d